In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import glob
from sklearn.externals import joblib
from scipy.ndimage.measurements import label
from lesson_functions import *
from model_parameters import *

In [ ]:
classifier = joblib.load('classifier.pkl')
svc = classifier['svc']
X_scaler = classifier['X_scaler']
print('Model loaded')

In [ ]:
images = glob.glob('/Users/tsekityam/git/CarND-Vehicle-Detection/test_images/test*.jpg')

fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    image = mpimg.imread(fname)
    heat = np.zeros_like(image[:,:,0]).astype(np.float)

    scales = np.linspace(1.0, 2.5, num=7)
    bbox_list = find_cars(image, y_start_stop[0], y_start_stop[1], scales, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, color_space=color_space, hog_channel=hog_channel,
              spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)                    

    # Add heat to each box in box list
    heat = add_heat(heat,bbox_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,1)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    sub = plt.subplot(len(images), 1, idx+1)
    sub.imshow(draw_img)

fig.tight_layout()
plt.show()